In [8]:
from src.utils.detect import Detect
detector = Detect('./resource/test/test2.png')
components = detector.get_valid_components()

In [9]:
print(components)

[{'index': 9, 'type': 'android.widget.ImageButton', 'content': '', 'resource_id': '', 'bounds': (0, 127, 158, 295)}, {'index': 10, 'type': 'android.widget.TextView', 'content': 'Manage storage', 'resource_id': '', 'bounds': (158, 172, 575, 249)}, {'index': 17, 'type': 'android.widget.LinearLayout', 'content': '', 'resource_id': 'com.android.settings:id/as_usage_summary_container', 'bounds': (130, 295, 768, 401)}, {'index': 20, 'type': 'android.widget.LinearLayout', 'content': '', 'resource_id': 'com.android.settings:id/as_usage_rate_container', 'bounds': (130, 401, 768, 550)}, {'index': 23, 'type': 'android.widget.LinearLayout', 'content': '', 'resource_id': 'com.android.settings:id/space_size_container', 'bounds': (1420, 382, 1726, 439)}, {'index': 26, 'type': 'android.widget.LinearLayout', 'content': '', 'resource_id': 'com.android.settings:id/as_usage_progress_container', 'bounds': (768, 439, 1726, 550)}, {'index': 34, 'type': 'android.widget.LinearLayout', 'content': '', 'resource_

In [10]:
for comp in components:
    print(comp['bounds'])

(0, 127, 158, 295)
(158, 172, 575, 249)
(130, 295, 768, 401)
(130, 401, 768, 550)
(1420, 382, 1726, 439)
(768, 439, 1726, 550)
(130, 550, 928, 623)
(928, 550, 1726, 623)
(130, 623, 928, 696)
(928, 623, 1726, 696)
(1365, 722, 1689, 817)
(198, 994, 454, 1089)
(454, 994, 762, 1089)
(878, 1217, 978, 1235)
(193, 1340, 288, 1435)
(1520, 1346, 1663, 1390)
(193, 1524, 288, 1619)
(1615, 1530, 1663, 1574)
(193, 1708, 288, 1803)
(1615, 1714, 1663, 1758)
(193, 1892, 288, 1987)
(1552, 1898, 1663, 1942)


In [42]:
sorted_components = sorted(components, key=lambda x: x['bounds'][1])
for comp in sorted_components:
    print(comp['bounds'])

print(len(sorted_components))

(0, 127, 158, 295)
(158, 172, 575, 249)
(130, 295, 768, 401)
(1420, 382, 1726, 439)
(130, 401, 768, 550)
(768, 439, 1726, 550)
(130, 550, 928, 623)
(928, 550, 1726, 623)
(130, 623, 928, 696)
(928, 623, 1726, 696)
(1365, 722, 1689, 817)
(198, 994, 454, 1089)
(454, 994, 762, 1089)
(878, 1217, 978, 1235)
(193, 1340, 288, 1435)
(1520, 1346, 1663, 1390)
(193, 1524, 288, 1619)
(1615, 1530, 1663, 1574)
(193, 1708, 288, 1803)
(1615, 1714, 1663, 1758)
(193, 1892, 288, 1987)
(1552, 1898, 1663, 1942)
22


In [50]:
def check_overlap(groups, vertical_bound):
    """
    두 범위가 겹치는지 확인하는 함수
    겹치는 경우: max(start1, start2) < min(end1, end2)
    """
    overlapping_groups = []
    
    for i, group in enumerate(groups):
        existing_start, existing_end = group['bounds']
        new_start, new_end = vertical_bound
        
        # 두 범위가 겹치는지 확인
        if max(existing_start, new_start) < min(existing_end, new_end):
            overlapping_groups.append(i)
    
    return len(overlapping_groups), overlapping_groups


groups = []
for comp in sorted_components:
    vertical_bound = (comp['bounds'][1], comp['bounds'][3])
    
    if not groups:
        groups.append({"bounds": vertical_bound, "components": [comp['bounds']]})
        continue
    
    overlap_count, overlapping_indices = check_overlap(groups, vertical_bound)
    
    if overlap_count == 0:
        # 겹치지 않는 경우 새로운 그룹 생성
        groups.append({"bounds": vertical_bound, "components": [comp['bounds']]})
    
    elif overlap_count == 1:
        # 하나의 그룹과 겹치는 경우 해당 그룹에 추가하고 범위 확장
        idx = overlapping_indices[0]
        existing_start, existing_end = groups[idx]['bounds']
        new_start, new_end = vertical_bound
        
        # 범위 확장 (최소값과 최대값으로)
        groups[idx]['bounds'] = (min(existing_start, new_start), max(existing_end, new_end))
        groups[idx]['components'].append(comp['bounds'])
    
    else:
        # 여러 그룹과 겹치는 경우 - 모든 겹치는 그룹을 병합
        all_components = [comp]
        min_start = vertical_bound[0]
        max_end = vertical_bound[1]
        
        # 역순으로 제거해야 인덱스 꼬임 방지
        for idx in sorted(overlapping_indices, reverse=True):
            all_components.extend(groups[idx]['components'])
            existing_start, existing_end = groups[idx]['bounds']
            min_start = min(min_start, existing_start)
            max_end = max(max_end, existing_end)
            groups.pop(idx)
        
        # 새로운 병합된 그룹 추가
        groups.append({"bounds": (min_start, max_end), "components": all_components})

print(groups)

[{'bounds': (127, 295), 'components': [(0, 127, 158, 295), (158, 172, 575, 249)]}, {'bounds': (295, 550), 'components': [(130, 295, 768, 401), (1420, 382, 1726, 439), (130, 401, 768, 550), (768, 439, 1726, 550)]}, {'bounds': (550, 623), 'components': [(130, 550, 928, 623), (928, 550, 1726, 623)]}, {'bounds': (623, 696), 'components': [(130, 623, 928, 696), (928, 623, 1726, 696)]}, {'bounds': (722, 817), 'components': [(1365, 722, 1689, 817)]}, {'bounds': (994, 1089), 'components': [(198, 994, 454, 1089), (454, 994, 762, 1089)]}, {'bounds': (1217, 1235), 'components': [(878, 1217, 978, 1235)]}, {'bounds': (1340, 1435), 'components': [(193, 1340, 288, 1435), (1520, 1346, 1663, 1390)]}, {'bounds': (1524, 1619), 'components': [(193, 1524, 288, 1619), (1615, 1530, 1663, 1574)]}, {'bounds': (1708, 1803), 'components': [(193, 1708, 288, 1803), (1615, 1714, 1663, 1758)]}, {'bounds': (1892, 1987), 'components': [(193, 1892, 288, 1987), (1552, 1898, 1663, 1942)]}]


In [48]:
components = [{'index': 17, 'type': 'android.widget.LinearLayout', 'content': '', 'resource_id': 'com.android.settings:id/as_usage_summary_container', 'bounds': (130, 295, 768, 401)}, 
{'index': 23, 'type': 'android.widget.LinearLayout', 'content': '', 'resource_id': 'com.android.settings:id/space_size_container', 'bounds': (1420, 382, 1726, 439)}, 
{'index': 20, 'type': 'android.widget.LinearLayout', 'content': '', 'resource_id': 'com.android.settings:id/as_usage_rate_container', 'bounds': (130, 401, 768, 550)}, 
{'index': 26, 'type': 'android.widget.LinearLayout', 'content': '', 'resource_id': 'com.android.settings:id/as_usage_progress_container', 'bounds': (768, 439, 1726, 550)}]

components = sorted(components, key=lambda x: x['bounds'][0])

for comp in components:
    print(comp['bounds'])






(130, 295, 768, 401)
(130, 401, 768, 550)
(768, 439, 1726, 550)
(1420, 382, 1726, 439)
